In [1]:
"""
make PySDM files accessible to import statements
"""

import sys, os
if 'google.colab' in sys.modules:  # To use GPU on colab set hardware accelerator as 'GPU' before start session
    ! git clone --recurse-submodules -j8 https://github.com/piotrbartman/PySDM.git
    %cd PySDM
    ! pip install -U $(cat requirements.txt | cut -d '=' -f 1)
    ! pip install -U $(cat submodules/MPyDATA/requirements.txt | cut -d '=' -f 1)
    ! pip install submodules/MPyDATA
else:
    sys.path.append(os.path.join(os.getcwd(), '../..'))

In [2]:
from numpy import errstate
from PySDM.backends import Default, ThrustRTC
from PySDM_examples.Shima_et_al_2009_Fig_2.spectrum_plotter import SpectrumPlotter
from PySDM_examples.Shima_et_al_2009_Fig_2.setup import SetupA
from PySDM_examples.Shima_et_al_2009_Fig_2.example import run
import ipywidgets as widgets

In [6]:
prwidgets.IntProgress(min=0, max=100, description='%')
class ProgbarUpdater:
    def __init__(self, progbar, max_steps):
        self.max_steps = max_steps
        self.steps = 0
        self.progbar = progbar

    def notify(self):
        self.steps += 1
        self.progbar.value = 100 * (self.steps / self.max_steps)

In [7]:
def demo(*, n_SD, n_step, n_plot, adaptive, gpu):
    with errstate(all='raise'):
        setup = SetupA()
        setup.backend = ThrustRTC() if gpu else Default()
        setup.n_sd = 2 ** n_SD
        setup.adaptive = adaptive
        setup._steps = [i * (n_step // n_plot) for i in range(n_plot + 1)]
        
        states, _ = run(setup, (ProgbarUpdater(progbar, setup._steps[-1]),))
    
    with errstate(invalid='ignore'):
        plotter = SpectrumPlotter(setup)
        plotter.smooth = True
        for step, state in states.items():
            plotter.plot(state, step * setup.dt)
        plotter.show()

In [8]:
n_SD = widgets.IntSlider(value=14, min=12, max=18, step=1, description='$log_2(n_{SD})$', continuous_update=False)
n_step = widgets.IntSlider(value=3600, step=100, min=0, max=3600, description='$n_{step}$', continuous_update=False)
n_plot = widgets.IntSlider(value=3, step=1, min=1, max=8, description='$n_{plot}$', continuous_update=False)
sliders = widgets.HBox([n_SD, n_step, n_plot])

adaptive = widgets.Checkbox(value=False, description='adaptive dt')
gpu = widgets.Checkbox(value=False, description='GPU')
options = [adaptive]
if ThrustRTC.ENABLE:
    options.append(gpu)
boxes = widgets.HBox(options)

self = widgets.interactive_output(demo, 
                                  {'n_SD': n_SD, 'n_step': n_step, 'n_plot': n_plot, 'adaptive': adaptive, 'gpu': gpu})

display(sliders, boxes, progbar, self)

NameError: name 'progbar' is not defined